In [1]:
%%capture
!pip install transformers
!git clone https://github.com/ronakdm/input-marginalization.git

In [2]:
%%bash
cd input-marginalization
git pull
cd ..

Updating 0c9c57a..65cdab4
Fast-forward
 metrics.py | 8 ++++----
 1 file changed, 4 insertions(+), 4 deletions(-)


From https://github.com/ronakdm/input-marginalization
   0c9c57a..65cdab4  main       -> origin/main


In [3]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
save_dir = "/content/gdrive/My Drive/input-marginalization"

Mounted at /content/gdrive


In [4]:
%%capture
import sys
sys.path.append("input-marginalization")

from metrics import input_marginalization, continuous_colored_sentence
import torch
from transformers import BertForMaskedLM
import pickle

In [5]:
lstm_sentences = [
  ("pacino is brilliant as the sleep - deprived dormer , his increasing weariness as much existential as it is physical .", 1),
  ("an important movie , a reminder of the power of file to move us and to make us examine our values .", 1),
  ("unflinchingly bleak and desperate", 0),
  ("i am sorry that i was unable to get the full brunt of this comedy .", 0),
]
bert_sentences = [
  ("it ' s a lovely film with lovely performances by buy and accorsi .", 1),
  ("more romantic , emotional and ultimately more satisfying than the teary - eyed original .", 1),
  ("it ' s a bit disappointing that it only manages to be decent instead of dead brilliant .", 0),
  ("suffers from the lack of a compelling or comprehensible narrative .", 0),   
]
cnn_sentences = [
  (". . . very funny , very enjoyable . . .", 1),
  ("lookin ' for sin , american - style ?", 0),
]

These cells compute the attribution scores. If they are already computed, skip ahead to where they are displayed.

In [5]:
%%capture
mlm = BertForMaskedLM.from_pretrained('bert-base-uncased').cuda()

In [8]:
lstm = torch.load(f"{save_dir}/lstm_sst2.pt")

lstm_att_scores = []
for sentence, label in lstm_sentences:
  lstm_att_scores.append(input_marginalization(lstm, sentence, mlm, target_label=label))
pickle.dump(lstm_att_scores, open(f"{save_dir}/lstm_att_scores_fig2.p", "wb" ))

In [ ]:
try:
  bert = torch.load(f"{save_dir}/bert_sst2.pt")

  bert_att_scores = []
  for sentence, label in bert_sentences:
    bert_att_scores.append(input_marginalization(bert, sentence, mlm, target_label=label))
  
  pickle.dump(bert_att_scores, open(f"{save_dir}/bert_att_scores_fig2.p", "wb" ))
except KeyboardInterrupt:
  print("Graceful Exit")

Graceful Exit


In [12]:
cnn = torch.load(f"{save_dir}/cnn_sst2.pt")

cnn_att_scores = []
for sentence, label in cnn_sentences:
  cnn_att_scores.append(input_marginalization(cnn, sentence, mlm, target_label=label))
pickle.dump(cnn_att_scores, open(f"{save_dir}/cnn_att_scores_fig2.p", "wb" ))

Colorized sentences displayed below.

In [20]:
lstm_att_scores = pickle.load(open(f"{save_dir}/lstm_att_scores_fig2.p", "rb" ))
for i, pair in enumerate(lstm_sentences):
  continuous_colored_sentence(pair[0], lstm_att_scores[i], color_limit=0.003)
  print("")

 pacino is brilliant as the sleep - deprived dormer , his increasing weariness as much existential as it is physical .

 an important movie , a reminder of the power of file to move us and to make us examine our values .

 unflinchingly bleak and desperate

 i am sorry that i was unable to get the full brunt of this comedy .



In [18]:
cnn_att_scores = pickle.load(open(f"{save_dir}/cnn_att_scores_fig2.p", "rb" ))
for i, pair in enumerate(cnn_sentences):
  continuous_colored_sentence(pair[0], cnn_att_scores[i], color_limit=0.0003)
  print("")

 . . . very funny , very enjoyable . . .

 lookin ' for sin , american - style ?

